In [1]:
from models.PlaceNet import PlaceNet
from helpers import logger, visualize_segmentation
from datasets import SegmentationDataset
import logging
import torch

logger.setLevel(logging.INFO)

In [2]:
model = PlaceNet()

In [3]:
dummy_in_full = torch.ones((1,7,480,640))
dummy_in_half = torch.ones((1,7,240,320))
dummy_in_quarter = torch.ones((1,7,120,160))

recon_full, recon_half, recon_quarter = model(dummy_in_full, dummy_in_half, dummy_in_quarter)

print("-"*100)
print(recon_full.shape)
print(recon_half.shape)
print(recon_quarter.shape)

----------------------------------------------------------------------------------------------------
torch.Size([1, 7, 480, 640])
torch.Size([1, 7, 240, 320])
torch.Size([1, 7, 120, 160])


In [6]:
from helpers import WMSELoss

loss = WMSELoss(recon_full, dummy_in_full)
loss

tensor(2.0023, grad_fn=<AddBackward0>)

In [7]:
from transformers import AutoImageProcessor, UperNetForSemanticSegmentation
from torch.utils.data import DataLoader

# Initialize processor and model
model_base = "openmmlab/upernet-swin-large"
processor = AutoImageProcessor.from_pretrained(model_base)
model = UperNetForSemanticSegmentation.from_pretrained(model_base)


/opt/homebrew/Caskroom/miniforge/base/envs/mtpqt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

/opt/homebrew/Caskroom/miniforge/base/envs/mtpqt/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/8.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/940M [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
# Replace this with the path to your dataset of images
image_dir = ""

# Create dataset and dataloader
dataset = SegmentationDataset(image_dir, processor)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

# Process each image in the dataset
for image_np, pixel_values in dataloader:
    with torch.no_grad():
        outputs = model(pixel_values)
        seg = processor.post_process_semantic_segmentation(outputs, target_sizes=[image_np.shape[1:3]])[0]
    
    # Visualize the segmentation
    visualize_segmentation(image_np.squeeze(0), seg)